In [1]:
# Enable autoreload of module
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
import json
import os
from data.DWSNets_dataset import DWSNetsDataset
from networks.regression_transformer import RegressionTransformerConfig
from training.naive_approach import train, Config


In [2]:
# Config
config = Config()
model_config = RegressionTransformerConfig(n_embd=1, block_size=1024, n_head=1)


device = config.device

In [3]:
# Dataloading
import torch.utils


dir_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(dir_path, "adl4cv", "datasets", "DWSNets", "mnist-inrs")

dataset = DWSNetsDataset(data_root)
path = "datasets/DWSNets/mnist-inrs/mnist_splits.json"
dataset_map = json.load(open(path, "r"))


/Users/luca/uni/master/second-semester/adl4cv/datasets/DWSNets/mnist-inrs/mnist_splits.json


In [4]:
from collections import OrderedDict

def flatten_weights(weights: OrderedDict) -> torch.Tensor:
    """
    Flatten the weights of a model
    :param weights: Ordered dictionary of weights (string -> tensor)
    :return: The flattened weights (single tensor)
    """
    return torch.cat([torch.flatten(v) for k, v in weights.items()])


weights = torch.load(dataset_map["train"]["path"][0], map_location=torch.device('cpu'))
flattened = flatten_weights(weights)


def get_batch(split: str):
    # let's get a batch with the single element
    # y should be the same shifted by 1
    ix = torch.randint(torch.numel(flattened) - model_config.block_size, (config.batch_size,))
    x = torch.stack(
        [flattened[i : i + model_config.block_size] for i in ix]
    )
    y = torch.stack(
        [flattened[i + 1 : i + 1 + model_config.block_size] for i in ix]
    )
    # x and y have to be (1, *, 1)
    x = x.unsqueeze(-1)
    y = y.unsqueeze(-1)
    return x, y

In [7]:
# Prepeare model parameters and train
train(get_batch, config, model_config)

Initializing a new model from scratch
num decayed parameter tensors: 50, with 1,169 parameters
num non-decayed parameter tensors: 98, with 158 parameters
using fused AdamW: False


/Users/luca/uni/master/second-semester/adl4cv/.venv/lib/python3.12/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


iter,▁▁▁▁▁▁▁
lr,▁▁▁▁▁▁▁
mfu,▁▁▁▁▁▁▁
train/loss,▁▆▆▇▄█▄
val/loss,▄▁█▁▂▂▃
iter,0
lr,0.0
mfu,-100.0
train/loss,0.00035
val/loss,0.00036


step 0: train loss 0.0004, val loss 0.0004
step 2000: train loss 0.0004, val loss 0.0003
saving checkpoint to models
step 4000: train loss 0.0003, val loss 0.0003
saving checkpoint to models


KeyboardInterrupt: 